In [1]:
import ctypes
import numpy as np
import json

In [2]:
a = np.load("triangles_a.npy")
b = np.load("triangles_b.npy")

In [3]:
ifortlib = ctypes.CDLL("./src/fortran/ifort_clipping.dll")
def ifort_area_of_intersection(a, b):
    ntriangles, nvertex, ndim = a.shape
    out = np.empty(ntriangles, dtype=np.float64)
    function = ifortlib.area_of_intersection
    function(
        ctypes.pointer(ctypes.c_longlong(ndim)),
        ctypes.pointer(ctypes.c_longlong(nvertex)),
        ctypes.pointer(ctypes.c_longlong(ntriangles)),
        np.ctypeslib.as_ctypes(a),
        np.ctypeslib.as_ctypes(b),
        np.ctypeslib.as_ctypes(out),
    )
    return out

gfortranlib = ctypes.CDLL("./src/fortran/gfortran_clipping.dll")
def gfortran_area_of_intersection(a, b):
    ntriangles, nvertex, ndim = a.shape
    out = np.empty(ntriangles, dtype=np.float64)
    function = gfortranlib.area_of_intersection
    function(
        ctypes.pointer(ctypes.c_longlong(ndim)),
        ctypes.pointer(ctypes.c_longlong(nvertex)),
        ctypes.pointer(ctypes.c_longlong(ntriangles)),
        np.ctypeslib.as_ctypes(a),
        np.ctypeslib.as_ctypes(b),
        np.ctypeslib.as_ctypes(out),
    )
    return out

In [4]:
area_of_intersection = ifort_area_of_intersection

times = {}
for s in [1, 10, 100, 1_000, 10_000, 100_000, 1_000_000, 10_000_000]:
    res = %timeit -o area_of_intersection(a[:s], b[:s])
    times[s] = res.average
    
json.dump(times, open("timing/python-ifort.json", "w"))
np.save("answers/python-ifort.npy", area_of_intersection(a[:10_000], b[:10_000]))

27.2 µs ± 622 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
27.7 µs ± 243 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
36.2 µs ± 918 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
138 µs ± 3.76 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.16 ms ± 4.49 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
11.3 ms ± 78.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
117 ms ± 1.73 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.17 s ± 6.27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
area_of_intersection = gfortran_area_of_intersection

times = {}
for s in [1, 10, 100, 1_000, 10_000, 100_000, 1_000_000, 10_000_000]:
    res = %timeit -o area_of_intersection(a[:s], b[:s])
    times[s] = res.average
    
json.dump(times, open("timing/python-gfortran.json", "w"))
np.save("answers/python-gfortran.npy", area_of_intersection(a[:10_000], b[:10_000]))

28.1 µs ± 480 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
29.7 µs ± 258 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
48.5 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
244 µs ± 2.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
2.16 ms ± 18.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
21.6 ms ± 686 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
221 ms ± 2.98 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.18 s ± 25.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
